In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn import metrics
import hdbscan
import umap
import umap.plot

In [3]:
df = pd.read_csv('dataset/semantic_similarity/HSCode_2022.csv')

In [4]:
df

,HSCodeNo,HSCode2,HSCode4,HSCode6,HSCode8,HSCode11,endescriptions,thdescriptions,HSYear,endescriptions2,...,HSCodeEnDescriptions2,HSCodeTHDescriptions2,HSCodeEnDescriptions4,HSCodeTHDescriptions4,HSCodeEnDescriptions6,HSCodeTHDescriptions6,HSCodeEnDescriptions8,HSCodeTHDescriptions8,HSCodeEnDescriptions11,HSCodeTHDescriptions11
0,28793.0,20,2008,200899,20089910,20089910090,Other ...,อื่น ๆ ...,2022,"Preparations of vegetables, fruit, nuts or oth...",...,"20:Preparations of vegetables, fruit, nuts or ...",20:ของปรุงแต่งทำจากพืชผัก ผลไม้ ลูกนัต หรือจาก...,"2008:Fruit, nuts and other edible parts of pla...",2008:ผลไม้ ลูกนัต และส่วนอื่นที่บริโภคได้ของพื...,200899:Other,200899:อื่น ๆ,20089910:Lychees,20089910:ลิ้นจี่,20089910090:Other ...,20089910090:อื่น ๆ ...
1,76015.0,87,8703,870332,87033235,87033235999,"Of a cylinder capacity exceeding 1,800 cc but ...","ความจุของกระบอกสูบเกิน 1,800 ลูกบาศก์ เซนติเมต...",2022,Tractors (other than tractors of heading 87.09).,...,87:Tractors (other than tractors of heading 87...,87:แทรกเตอร์(นอกจากแทรกเตอร์ตามประเภท 87.09),8703:Motor cars and other motor vehicles princ...,8703:รถยนต์และยานยนต์อื่นๆ ที่ออกแบบสำหรับขนส่...,"870332:Of a cylinder capacity exceeding 1,500 ...","870332:ความจุของกระบอกสูบเกิน 1,500 ลูกบาศก์เซ...","87033235:Of a cylinder capacity exceeding 1,80...","87033235:ความจุของกระบอกสูบเกิน 1,800 ลูกบาศก์...",87033235999:Of a cylinder capacity exceeding 1...,"87033235999:ความจุของกระบอกสูบเกิน 1,800 ลูกบา..."
2,43610.0,38,3808,380891,38089199,38089199544,N-sec-BUTYL PHTHALIMIDE ...,เอ็น-เซคัลดารี-บิวทิลฟทาลิไมด์ ...,2022,Miscellaneous chemical products,...,38:Miscellaneous chemical products,38:เคมีภัณฑ์เบ็ดเตล็ด,"3808:Insecticides, rodenticides, fungicides, h...",3808:สารฆ่าแมลง สารฆ่าสัตว์แทะ สารฆ่ารา สารฆ่า...,380891:Insecticides,380891:สารฆ่าแมลง,38089199:Other,38089199:อื่น ๆ,38089199544:N-sec-BUTYL PHTHALIMIDE ...,38089199544:เอ็น-เซคัลดารี-บิวทิลฟทาลิไมด์ ...
3,71701.0,85,8501,850163,85016300,85016300000,Of an output exceeding 375 kVA but not exceedi...,ให้กำลังเกิน 375 เควีเอ แต่ไม่เกิน 750 เควีเอ ...,2022,"Machinery, electrical appliances And component...",...,"85:Machinery, electrical appliances And compon...",85:เครื่องจักร เครื่องอุปกรณ์ไฟฟ้า และส่วนประก...,8501:Electric motors and generators (excluding...,8501:มอเตอร์ไฟฟ้าและเครื่องกำเนิดไฟฟ้า (ไม่รวม...,850163:Of an output exceeding 375 kVA but not ...,850163:ให้กำลังเกิน 375 เควีเอ แต่ไม่เกิน 750 ...,85016300:Of an output exceeding 375 kVA but no...,85016300:ให้กำลังเกิน 375 เควีเอ แต่ไม่เกิน 75...,85016300000:Of an output exceeding 375 kVA but...,85016300000:ให้กำลังเกิน 375 เควีเอ แต่ไม่เกิน...
4,41093.0,38,3808,380891,38089120,38089120590,SODIUM FLUORIDE ...,โซเดียม ฟลูออไรด์ ...,2022,Miscellaneous chemical products,...,38:Miscellaneous chemical products,38:เคมีภัณฑ์เบ็ดเตล็ด,"3808:Insecticides, rodenticides, fungicides, h...",3808:สารฆ่าแมลง สารฆ่าสัตว์แทะ สารฆ่ารา สารฆ่า...,380891:Insecticides,380891:สารฆ่าแมลง,"38089120:Other, powder for moulding into mosqu...",38089120:อื่น ๆ เป็นผงสำหรับผสมในขดกันยุง,38089120590:SODIUM FLUORIDE ...,38089120590:โซเดียม ฟลูออไรด์ ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,33593.0,29,2912,291260,29126000,29126000000,Paraformaldehyde ...,พาราฟอร์มัลดีไฮด์ ...,2022,Organic chemicals,...,29:Organic chemicals,29:เคมีภัณฑ์อินทรีย์,"2912:Aldehydes, whether or not with other oxyg...",2912:แอลดีไฮด์ จะมีออกซิเจนฟังก์ชั่นอื่นด้วยหร...,291260:Paraformaldehyde ...,291260:พาราฟอร์มัลดีไฮด์ ...,29126000:Paraformaldehyde ...,29126000:พาราฟอร์มัลดีไฮด์ ...,29126000000:Paraformaldehyde ...,29126000000:พาราฟอร์มัลดีไฮด์ ...
298,80427.0,94,9405,940530,94053000,94053000000,NaN,NaN,2022,"Furniture ; bedding, mattresses, mattress supp...",...,"94:Furniture ; bedding, mattresses, mattress s...",94:เฟอร์นิเจอร์ เครื่องเตียง ฟูก ฐานรองฟูก เบา...,9405:Lamps and lighting fittings including sea...,9405:เครื่องประทีปโคมไฟรวมถึงเซิร์ชไลต์ สปอร์ต...,94053

In [5]:
df.columns

Index(['HSCodeNo', 'HSCode2', 'HSCode4', 'HSCode6', 'HSCode8', 'HSCode11',
       'endescriptions', 'thdescriptions', 'HSYear', 'endescriptions2',
       'thdescriptions2', 'endescriptions4', 'thdescriptions4',
       'endescriptions6', 'thdescriptions6', 'endescriptions8',
       'thdescriptions8', 'endescriptions11', 'thdescriptions11',
       'HSCodeEnDescriptions2', 'HSCodeTHDescriptions2',
       'HSCodeEnDescriptions4', 'HSCodeTHDescriptions4',
       'HSCodeEnDescriptions6', 'HSCodeTHDescriptions6',
       'HSCodeEnDescriptions8', 'HSCodeTHDescriptions8',
       'HSCodeEnDescriptions11', 'HSCodeTHDescriptions11'],
      dtype='object')

In [6]:
df['thdescriptions2']

0      ของปรุงแต่งทำจากพืชผัก ผลไม้ ลูกนัต หรือจากส่ว...
1              แทรกเตอร์(นอกจากแทรกเตอร์ตามประเภท 87.09)
2                                     เคมีภัณฑ์เบ็ดเตล็ด
3      เครื่องจักร เครื่องอุปกรณ์ไฟฟ้า และส่วนประกอบข...
4                                     เคมีภัณฑ์เบ็ดเตล็ด
                             ...                        
297                                    เคมีภัณฑ์อินทรีย์
298    เฟอร์นิเจอร์ เครื่องเตียง ฟูก ฐานรองฟูก เบาะแล...
299    ไขมันและน้ำมันที่ได้จากสัตว์หรือพืช และผลิตภัณ...
300                                               ธัญพืช
301                                    เคมีภัณฑ์อินทรีย์
Name: thdescriptions2, Length: 302, dtype: object

In [7]:
df['thdescriptions4']

0      ผลไม้ ลูกนัต และส่วนอื่นที่บริโภคได้ของพืชที่ป...
1      รถยนต์และยานยนต์อื่นๆ ที่ออกแบบสำหรับขนส่งบุคค...
2      สารฆ่าแมลง สารฆ่าสัตว์แทะ สารฆ่ารา สารฆ่าวัชพื...
3      มอเตอร์ไฟฟ้าและเครื่องกำเนิดไฟฟ้า (ไม่รวมถึงชุ...
4      สารฆ่าแมลง สารฆ่าสัตว์แทะ สารฆ่ารา สารฆ่าวัชพื...
                             ...                        
297    แอลดีไฮด์ จะมีออกซิเจนฟังก์ชั่นอื่นด้วยหรือไม่...
298    เครื่องประทีปโคมไฟรวมถึงเซิร์ชไลต์ สปอร์ตไลต์ ...
299    ไขมันหรือน้ำมันที่ได้จากสัตว์หรือพืชและเศษของข...
300                                                 ข้าว
301    สารประกอบเฮตเทอโรไซคลิกที่มีไนโตรเจนเฮตเทอโรอะ...
Name: thdescriptions4, Length: 302, dtype: object

In [37]:
sentences_text = np.concatenate((
    df['thdescriptions2'].unique(), 
    df['thdescriptions4'].unique(),
    df['endescriptions2'].unique(),
    df['endescriptions4'].unique(),
))
sentences_text.shape

(495,)

In [47]:
df_sentences = pd.DataFrame(sentences_text, columns=['text'])
df_sentences.shape

(495, 1)

In [48]:
df_sentences.sample(20)

,text
488,"Fish, frozen, excluding fish fillets and other..."
40,ธัญพืช
395,Other garments (other than those specified of ...
100,ปลาที่ปรุงแต่งหรือทำไว้ไม่ให้เสีย คาเวียร์และข...
149,ไฮโดรเจน แรร์ก๊าซ และอโลหะอื่น ๆ
338,"Pile fabrics, including ""long pile"" fabrics an..."
112,เครื่องประมวลผลข้อมูลอัตโนมัติและหน่วยต่าง ๆ ข...
111,สารประกอบออร์แกโน อินออร์แกนิกอื่น ๆ
278,"Products of animal origin, not elsewhere speci..."
402,Pharmaceutical goods specified in Note 4 to th...


In [21]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [ ]:
X = model.encode(df_sentences['text'].values, convert_to_tensor=True)

In [52]:
X.shape

torch.Size([495, 512])

#### HDBSCAN + UMAP

In [108]:
X_umap = umap.UMAP(n_neighbors=5, n_components=5).fit_transform(X)
clusterer = hdbscan.HDBSCAN(alpha=1.0, min_cluster_size=5)
hdbscan_cluster = clusterer.fit_predict(X_umap)
hdbscan_cluster

array([ 5, 11, 22, 21, 21, 10, 22, -1, 10, 21, -1,  3, 10,  5, 10, 10, 10,
        9, 22,  9, 21, 10, -1,  9,  3,  9,  5,  2, 16,  2,  9, -1, 13, 17,
       20, 10,  1,  4,  1,  9, 13, -1, 21,  9,  5, -1,  0, 13,  6, 23,  9,
       10,  5, 10, -1, -1,  8, 23,  9,  9, 15,  5,  5, 12,  5, 20, 14, 10,
       19, -1, 20, -1,  1, -1, -1,  3, 10, 16, 19,  5,  9, 19, 21, -1, 13,
       20, 20, 20, 17,  9,  9, 10,  9, 21, 10,  4, 18, -1, 10,  9,  3,  9,
        5, 16, 20, 13, 16,  0, 16,  2, 10, 10, 21,  0, -1, 16,  9,  8, 16,
        9, 11, 20, 18, -1,  3, 16, 21, 17, 21, 21, 13, 20,  1, 19,  0, 13,
       21,  1,  4, 11,  1,  1,  0,  0,  1,  6, 12,  9, 16, 17, 17, 13,  5,
       -1, 23, 12, 13, 20,  9,  9,  5, 10,  5, 20,  5, 20, -1,  5,  6, 14,
       -1,  4,  9,  3, 13, 13,  9, 21, -1, 10,  4,  8,  5, 18,  5, 21, 16,
       16, 20,  8, -1, 21,  6, 21, 20, 16, 16,  3,  1,  9,  5,  9,  9, 19,
        5, 20, 11,  5, 17, 20, -1, 21,  5, 13, 16, 14, 19, 18, 21,  9,  3,
        9,  5, 21, 16, 10

##### silhouette score

In [109]:
hdbscan_silhouette = metrics.silhouette_score(X_umap, hdbscan_cluster, metric='euclidean')
hdbscan_silhouette

0.3612751

##### Example First 10 rows of each group

In [111]:
df_sentences['hdbscan_group'] = hdbscan_cluster
df_sentences.groupby('hdbscan_group').head(10).sort_values(by=['hdbscan_group'], ascending=False)[:60]

,text,hdbscan_group,dbscan_group
277,Man - made staple fibres,23,2
154,ผลิตภัณฑ์ทางเภสัชกรรมที่ระบุไว้ในหมายเหตุ 4 ใน...,23,2
298,Ceramic products,23,2
49,ผลิตภัณฑ์เซรามิก ...,23,2
57,ผลิตภัณฑ์เบ็ดเตล็ด,23,2
360,Other organo inorganic compounds.,22,2
267,Inorganic chemicals Organic compounds,22,2
255,Organic chemicals,22,2
251,Miscellaneous chemical products,22,2
355,Nitrile-function compounds.,22,2


#### DBSCAN + UMAP

In [114]:
X_umap = umap.UMAP(n_neighbors=5, n_components=5).fit_transform(X)
dbscan_cluster = DBSCAN(eps=0.5, min_samples=2).fit(X_umap)
dbscan_cluster.labels_

array([ 0,  1,  2,  3,  3,  4,  2,  2,  4,  3,  5,  6,  4,  0,  4,  4,  4,
        7,  2,  7,  3,  4,  8,  7,  6,  7,  0,  9, 10,  9,  7,  9, 10, 11,
        3,  4, 12, 13, 12,  7, 10, 14,  3,  7,  0,  9, 15, 10, 16,  2,  7,
       17,  0,  4,  4,  8, 18,  2, 19,  7, 20,  0,  0,  1,  0,  3,  3, 17,
       21,  2,  3, 22, 12,  5, 11,  6,  4, 10, 21,  0, -1, 21,  3, 23, 10,
        3,  3,  3, 21,  7,  7, 17,  7,  3,  4, 13, 22,  2,  4,  7,  6,  7,
        0, 10,  3, 10, 10, 15, 10,  9,  4, 17,  3, 15,  1, 10,  7, 18, 10,
        7,  1,  3, 22, 23,  6, 10,  3, 11,  3,  3, 10,  3, 12, 21, 15, 10,
        3, 12, 13,  1, 12, 12, 15, 15, 12, 16,  1,  7, 10, 11, 11, 10,  0,
       14,  2,  1, 10,  3,  7,  7,  0,  4,  0,  3,  0,  3, 24,  0, 16,  3,
       24, 13,  7,  6, 10, 10,  7,  3,  1, 17, 13, 18,  0, 22,  0,  3, 10,
       10,  3, 18, 10,  3, 16,  3,  3, 10, 10,  6, 12,  7,  0, 19,  7, 21,
        0,  3,  1,  0, 11,  3, 25,  3,  0, 10, 10,  3, 21,  3,  3, 19,  6,
        7,  0,  3, 10, 17

##### silhouette score

In [115]:
dbscan_silhouette = metrics.silhouette_score(X_umap, dbscan_cluster.labels_, metric='euclidean')
dbscan_silhouette

0.3839889

##### Example First 10 rows of each group

In [116]:
df_sentences['dbscan_group'] = dbscan_cluster.labels_
df_sentences.groupby('dbscan_group').head(10).sort_values(by=['dbscan_group'], ascending=False)[:60]

,text,hdbscan_group,dbscan_group
382,Flat-rolled products of iron or non-alloy stee...,-1,27
391,Flat-rolled products of iron or non-alloy stee...,-1,27
356,Flat-rolled products of iron or non-alloy stee...,-1,27
362,Flat-rolled products of iron or non-alloy stee...,-1,27
390,"Flat-rolled products of stainless steel, of a ...",-1,27
234,สิ่งก่อสร้าง (ไม่รวมถึงอาคารสำเร็จรูปตามประเภท...,-1,26
480,Structures (excluding prefabricated buildings ...,-1,26
210,ผ้าที่ใช้กับเตียง โต๊ะ ห้องน้ำและห้องครัว,-1,25
457,"Bed linen, table linen, toilet linen and kitch...",-1,25
414,"Carboys, bottles, flasks, jars, pots, phials, ...",-1,24
